        Ситуация:

У меня открыто 2 брокерских счета у Finam и Тинькофф.

Мне необходимо самостоятельно отчитываться прибыли/убытках от продажи валюты.

Расходы на покупку валюты необходимо рассчитать методом FIFO (First In First Out), то есть при продаже валюты, происходит продажа валюты, которая куплена раньше всего.

        Задача:

Для корректного расчета расходов на покупку валюты требуется поочередно обработать все сделки с валютой у брокера за все время.

Расчеты нужно произвести для каждого брокера отдельно.

        Действия:

Я написал три пользовательские функции:

- tinkoff_def - для обработки отчетов Тинькофф брокера (может быть использована без других функций)

- finam_def - для обработки отчетов брокера Finam (может быть использована без других функций)

- sell_def - функция для расчета прибыли/убытков от проведенных сделок (принимает на обработку датафрейм, полученный от работы функций tinkkoff_def или finam_def)

        Результат:

В дальнейшем расчет прибыли/убытков от операций на фондовом рынке не будет занимать значительного времени, так как расчеты автоматизированы.

Для каждой функции написана документация, код содержит пояснения к коду.

In [48]:
import pandas as pd
import re
import os
import warnings


warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.set_option('display.max_columns', 40)

# функция для обработки отчетов Тинькофф
def tinkoff_def(folder: str):
    '''
    Функция обрабатывает отчеты брокера Тинькофф.\n
    Формат отчетов: .xlsx образца 2022 г.\n
    Параметры:\n
    folder: str\n
        Папка с отчетами брокера (отдельная папка для брокера)\n
    Пример:\n
    tinkoff_def('C:/Projects/Налоги/Тинькофф/')
    '''
    # указываем путь к папке с отчетами
    tinkoff_folder = folder
    tinkoff_files = os.listdir(tinkoff_folder)
# задаем регулярное выражение для выделения сделок с валютой
    match = re.compile(r'_TOM$|_TOD$')

    # создаем пустой датафрейм для отчетов
    tinkoff_dfs = pd.DataFrame()
    # цикл для обработки файлов в папке
    for file in tinkoff_files:
        # Читаем файл Excel
        df = (
            pd.read_excel(tinkoff_folder + file, skiprows=7)
            .dropna(axis=0, how='all')
            .dropna(axis=1, how='all')
        )

        last_row = int(df[df.isin(['1.2 Информация о неисполненных сделках на конец отчетного периода']).any(axis=1)].index[0])
        rows = list(range(last_row))
        columns_lst = ['Номер сделки', 'Дата заключения', 'Время', 'Вид сделки', 'Сокращенное наименование', 'Цена за единицу', 'Количество', 'Сумма сделки', 'Валюта расчетов', 'Дата расчетов']

        # Первая часть таблицы
        df1 = (
            df.copy()
            [df.index.isin(rows) &          # оставляем только нужные разделы отчета
            ~df['Номер сделки'].isna()]     # оставляем несмещенные вправо данные
            [df.columns[:-1]]               # удаляем последний столбец (в нем нет данных)
            )

        # Вторая часть таблицы
        df2 = (
            df.copy()
            [df.index.isin(rows) &           # оставляем только нужные разделы отчета
            df['Номер сделки'].isna()][1:]   # оставляем смещенные вправо данные
            [df.columns[1:]]                 # удаляем первый столбец (в нем нет данных)
            .set_axis(df1.columns, axis=1)
        )

        col_list = list(pd.Series(df1.columns).replace('\n', '', regex=True)) # удаляем спец символы
        col_list_strip = [col.strip() for col in col_list]                    # удаляем лишние пробелы

        df_con = (
            pd.concat([df1, df2], ignore_index=True)              # объединяем датафремы
            .set_axis(col_list_strip, axis=1)                     # переименовываем столбцы
            .dropna(axis=1, how='all').dropna(axis=0, how='all')  # удаляем пустые столбцы/строки
        )

        # фильтруем полученный датафрейм
        df_t = (
            df_con[df_con['Сокращенное наименование'].str.contains(match) == True][columns_lst]
        )

        df_t['Дата заключения'] = pd.to_datetime(df_t[['Дата заключения', 'Время']].agg(' '. join, axis=1), dayfirst=True)
        df_t['Дата расчетов'] = pd.to_datetime(df_t['Дата расчетов'], dayfirst=True)
        df_t[['Цена за единицу', 'Сумма сделки']] = df_t[['Цена за единицу', 'Сумма сделки']].replace(',', '.', regex=True)

        new_data_types = {
            'Количество': 'int32',
            'Цена за единицу': 'float32',
            'Сумма сделки': 'float32'}

        df_t = (
            df_t
            .drop('Время', axis=1)
            .astype(new_data_types))

        tinkoff_dfs = pd.concat([tinkoff_dfs, df_t], ignore_index=True)
        print(f'{file} обработан')
    tinkoff_dfs['Сокращенное наименование'] = tinkoff_dfs['Сокращенное наименование'].replace(match, '', regex=True)
    return (tinkoff_dfs)


# функция для обработки отчетов Finam
def finam_def(folder: str):
    '''
    Функция обрабатывает отчеты брокера Finam.\n
    Формат отчетов: .html образца 2022 г.\n
    Параметры:\n
    folder: str\n
        Папка с отчетами брокера (отдельная папка для брокера)\n
    Пример:\n
    finam_def('C:/Projects/Налоги/Finam/')
    '''
    # указываем путь к папке с отчетами
    finam_folder = folder
    finam_files = os.listdir(finam_folder)
    # задаем регулярное выражение для выделения сделок с валютой
    html_match = re.compile(r'_TOM|_TOD')

    # создаем пустой датафрейм для отчетов
    finam_dfs = pd.DataFrame()

    df_new_columns = ['Номер сделки', 'Дата заключения', 'Время', 'Вид сделки', 'Сокращенное наименование',
                            'Цена за единицу', 'Количество', 'Сумма сделки', 'Валюта расчетов', 'Дата расчетов']

    # цикл для обработки файлов в папке
    for file in finam_files:
        # условие для исключения обработки лишних файлов
        if 'html' not in file:
            continue
        # код для обработки файлов
        try:
            # читаем файл
            df = (
                pd.read_html(finam_folder + file, match=html_match, thousands=' ', decimal=',')[0] #, encoding='utf-8'
                [['№ сделки', 'Дата сделки', 'Время сделки', 'Вид сделки', 'Инструмент', 'Цена',
                        'Количество (шт.)', 'Объём сделки', 'Валюта котирования', 'Дата исполнения (факт)']]
                .set_axis(df_new_columns, axis = 1)
            )

            # оставляем только операции с валютой
            df = df[df['Сокращенное наименование'].str.contains(html_match) == True]
            # исправляем тип данных
            df['Валюта расчетов'] = df['Валюта расчетов'].replace('Рубль', 'RUB')
            df['Дата заключения'] = pd.to_datetime(df[['Дата заключения', 'Время']].agg(' '. join, axis=1), dayfirst=True)
            df['Дата расчетов'] = pd.to_datetime(df['Дата расчетов'], dayfirst=True)

            new_data_types = {
                'Количество': 'int32',
                'Цена за единицу': 'float32',
                'Сумма сделки': 'float32'}

            df_f = df.drop('Время', axis=1).astype(new_data_types)
            # объединяем полученные дата фреймы
            finam_dfs = pd.concat([finam_dfs, df_f], ignore_index=True)
            print(f'Файл {file} обработан')
        # продолжение цикла, если в файле нет нужный данных
        except ValueError as e:
            print(f'В файле {file} нет нужной таблицы!\n ValueError:', str(e))
            continue
    finam_dfs['Сокращенное наименование'] = finam_dfs['Сокращенное наименование'].replace(html_match, '', regex=True)
    return finam_dfs


# функция для расчета доходов и расходов по методу FIFO
def sell_def(folder: str, year: int):
    '''
    Warning: использовать только при совершении покупки и продажи валюты в рамках одного брокерского счета!\n
    Функция рассчитывает доходы/убытки от операций с валютой по методу FIFO.\n
    Расчеты должны производиться для кажного брокера отдельно.\n
    Форматы отчетов:\n
        Тинькофф - .xlsx образца 2022 г.\n
        Finam - .html образца 2022 г.\n
    Параметры:\n
    folder: str\n
        Папка с отчетами брокера (отдельная папка для брокера)\n
        Папка с отчетами должна содержать имя брокера: "Тинькофф" или "Finam".\n
    year: int\n
        Отчетный год\n
    Пример:\n
    sell_def('C:/Projects/Finam/', 2022)\n
    sell_def('C:/Projects/Тинькофф/', 2022)
    '''

    if 'finam' in folder.lower():
        df = finam_def(folder)
    elif 'тинькофф' in folder.lower():
        df = tinkoff_def(folder)

    # создаем лист с валютами
    currency_list = list(df['Сокращенное наименование'].unique())

    # создаем пустые датафреймы для отчетов
    all_buy = pd.DataFrame()
    all_sell = pd.DataFrame()

    # цикл для последовательного расчета стоимости покупок по методу FIFO для разных валют
    for currency in currency_list:
        # фильтруем датафрейм по валюте
        currency_df = df[df['Сокращенное наименование'] == currency]

        # создание таблицы с покупками
        df_buy = currency_df[currency_df['Вид сделки'] == 'Покупка'].copy()
        # индексы (с 0 до n) для таблицы
        ind_buy = pd.Index(range(0, df_buy.reset_index()['index'].count(), 1))
        # обновление индексов для таблицы с покупками
        df_buy = df_buy.set_index(ind_buy)

        # создание таблицы с продажами
        df_sell = currency_df[currency_df['Вид сделки'] == 'Продажа'].copy()
        # индексы (с 0 до n) для таблицы
        ind_sell = pd.Index(range(0, df_sell.reset_index()['index'].count(), 1))
        # обновление индексов для таблицы с продажами
        df_sell = df_sell.set_index(ind_sell)

        # создание столбца для себестоимости
        df_sell["Себестоимость покупки"] = 0
        # копия столбца с количеством проданных единиц
        df_sell["Sell_cnt"] = df_sell['Количество']

        # обрабатываем данные из таблиц
        # задаем изначальные значения индесков i, j для перебора строк, переменной для хранения стоимости покупок Buy_cost
        i = 0
        j = 0
        Buy_cost = 0
        # листы с индексами датафреймов
        index_sell = tuple(df_sell.index)
        index_buy = tuple(df_buy.index)
        # Обрабатываем датафреймы с покупками и продажами построчно. Цикл ограничен размерами массивов
        while i in index_sell and j in index_buy:
            # в цикле сравниваются 1 сделка по покупке и продаже валюты
            if df_sell.at[i, 'Sell_cnt'] > df_buy.at[j, 'Количество']:
                # количество проданных единиц больше количества купленных
                Buy_cost += df_buy.at[j, 'Количество']*df_buy.at[j, 'Цена за единицу']
                df_sell.at[i, 'Sell_cnt'] -= df_buy.at[j, 'Количество']
                df_buy.at[j, 'Количество'] = 0
                j += 1
            elif df_sell.at[i, 'Sell_cnt'] == df_buy.at[j, 'Количество']:
                # количество проданных единиц равно количеству купленных
                Buy_cost += df_buy.at[j, 'Количество']*df_buy.at[j, 'Цена за единицу']
                df_buy.at[j, 'Количество'] = 0
                df_sell.at[i, "Себестоимость покупки"] = round(Buy_cost, 2)
                Buy_cost = 0
                j += 1
                i += 1
            else:
                # количество проданных единиц меньше количества купленных
                Buy_cost += df_sell.at[i, 'Sell_cnt']*df_buy.at[j, 'Цена за единицу']
                df_buy.at[j, 'Количество'] -= df_sell.at[i, 'Sell_cnt']
                df_sell.at[i, "Себестоимость покупки"] = round(Buy_cost, 2)
                Buy_cost = 0
                i += 1

        # объединение полученных датафреймов в один
        all_buy = pd.concat([all_buy, df_buy], ignore_index=True)
        all_sell = pd.concat([all_sell, df_sell], ignore_index=True)

    # удаляем лишний столбец
    all_sell = all_sell.drop('Sell_cnt', axis=1)

    # фильтруем таблицу по выбранному году
    sell_year = all_sell[(all_sell['Дата расчетов'] >= f'{year}-01-01')
                         & (all_sell['Дата расчетов'] <= f'{year}-12-31')].copy()

    # считаем прибыль/убыток от сделок
    sell_year['Прибыль_убыток'] = sell_year['Сумма сделки'] - sell_year['Себестоимость покупки']

    return sell_year


In [56]:
import gdown

# скачиваем папки с отчетами с google drive

tinkoff_files_folder = 'https://drive.google.com/drive/folders/1azEai60SwVfN5p0wVo3bM_yQvGEZDYvG?usp=sharing'
finam_files_folder = 'https://drive.google.com/drive/folders/1I0z962kPezPbGi7wd1uUTaWki46XPb0J?usp=sharing'

gdown.download_folder(tinkoff_files_folder, quiet=True, remaining_ok=True)
gdown.download_folder(finam_files_folder, quiet=True, remaining_ok=True)


['e:\\Поиск работы\\Портфолио\\FIFO calculation\\Finam\\Черновол Денис Владимирович КЛФ-1463051 (01.10.2022 по 31.12.2022)_new.html',
 'e:\\Поиск работы\\Портфолио\\FIFO calculation\\Finam\\Черновол Денис Владимирович КЛФ-1463051 (06.07.2022 по 30.09.2022)_new.html']

In [57]:
finam_sell = sell_def('Finam/', 2022)


В файле Черновол Денис Владимирович КЛФ-1463051 (01.10.2022 по 31.12.2022)_new.html нет нужной таблицы!
 ValueError: No tables found matching pattern '_TOM|_TOD'
Файл Черновол Денис Владимирович КЛФ-1463051 (06.07.2022 по 30.09.2022)_new.html обработан


In [58]:
finam_sell


,Номер сделки,Дата заключения,Вид сделки,Сокращенное наименование,Цена за единицу,Количество,Сумма сделки,Валюта расчетов,Дата расчетов,Себестоимость покупки,Прибыль_убыток
0,513268000.0,2022-08-01 17:16:28,Продажа,USDRUB,60.3800,9000.0,543420.0,RUB,2022-08-01,553207.5,-9787.5
1,513402000.0,2022-08-02 10:02:22,Продажа,USDRUB,60.2425,1000.0,60242.5,RUB,2022-08-02,61467.5,-1225.0
2,513425000.0,2022-08-02 10:07:44,Продажа,USDRUB,60.1000,5000.0,300500.0,RUB,2022-08-02,301112.5,-612.5
3,513612000.0,2022-08-02 11:24:45,Продажа,USDRUB,60.1225,10000.0,601225.0,RUB,2022-08-02,601375.0,-150.0
4,513616000.0,2022-08-02 11:26:28,Продажа,USDRUB,60.1475,10000.0,601475.0,RUB,2022-08-03,601400.0,75.0
5,513403000.0,2022-08-02 10:03:03,Продажа,CNYRUB,8.9860,60000.0,539160.0,RUB,2022-08-02,538740.0,420.0
6,523729000.0,2022-09-01 10:11:47,Продажа,CNYRUB,8.7260,10000.0,87260.0,RUB,2022-09-02,89390.0,-2130.0


In [59]:
finam_sell.groupby('Сокращенное наименование').agg(
    {'Сумма сделки': sum, 'Себестоимость покупки': sum})


,Сумма сделки,Себестоимость покупки
Сокращенное наименование,,
CNYRUB,626420.0,628130.0
USDRUB,2106862.5,2118562.5


In [ ]:
sum(finam_sell['Сумма сделки']), sum(finam_sell['Себестоимость покупки'])


(2733282.5, 2746692.5)

In [ ]:
tinkoff_sell = sell_def('Тинькофф/', 2020)


broker-report-2019-09-01-2019-12-31.xlsx обработан
broker-report-2020-01-01-2020-12-31.xlsx обработан


In [ ]:
tinkoff_sell


,Номер сделки,Дата заключения,Вид сделки,Сокращенное наименование,Цена за единицу,Количество,Сумма сделки,Валюта расчетов,Дата расчетов,Себестоимость покупки,Прибыль_убыток
0,215133588,2019-12-30 12:21:23,Продажа,USDRUB,61.722500,1000,61722.500000,RUB,2020-01-09,63947.20,-2224.700000
1,219990323,2020-02-18 10:30:40,Продажа,USDRUB,63.762501,3,191.289993,RUB,2020-02-18,190.11,1.179993
2,229240134,2020-04-10 10:32:02,Продажа,USDRUB,73.852501,9,664.669983,RUB,2020-04-10,570.33,94.339983
3,229442079,2020-04-13 10:55:46,Продажа,USDRUB,73.547501,149,10958.580078,RUB,2020-04-13,9442.13,1516.450078
4,230692120,2020-04-20 11:32:57,Продажа,USDRUB,74.779999,114,8524.919922,RUB,2020-04-20,7224.18,1300.739922
5,232895898,2020-04-30 10:48:35,Продажа,USDRUB,72.665001,15,1089.979980,RUB,2020-04-30,950.55,139.429980
6,235452853,2020-05-19 10:29:12,Продажа,USDRUB,72.370003,150,10855.500000,RUB,2020-05-20,9505.50,1350.000000
7,237749500,2020-06-01 10:49:35,Продажа,USDRUB,69.720001,108,7529.759766,RUB,2020-06-01,6843.96,685.799766
8,262458445,2020-09-29 13:24:08,Продажа,USDRUB,79.107498,250,19776.880859,RUB,2020-09-30,15880.24,3896.640859
9,262723239,2020-09-30 11:01:52,Продажа,USDRUB,78.745003,100,7874.500000,RUB,2020-10-01,6388.00,1486.500000


In [ ]:
tinkoff_sell.groupby('Сокращенное наименование').agg(
    {'Сумма сделки': sum, 'Себестоимость покупки': sum})


,Сумма сделки,Себестоимость покупки
Сокращенное наименование,,
USDRUB,186724.953125,166171.3


In [ ]:
sum(tinkoff_sell['Сумма сделки']), sum(tinkoff_sell['Себестоимость покупки'])


(186724.96046447754, 166171.30000000005)